In [33]:
#so given a specific uhhhh uhhh pixel we wanna randomize values of 4 seperate pixels that average to that pixel? we can probably brute force this
#for now if we just wanna do the string of encrypted pixels from the previous program but will need to be reworked to have a smarter
#algorithm if we wanna scale up > will do if we have time

#imports
import random
import PIL
from PIL import Image 

def expand_pixel(des = [128,128,128,255], noise_buffer = 5): #should be passed an array with length 4 desired containing r,g,b,a values, default white because its easier to average to 255 and you cant really avg to black
    #note to matias itf we might have to use a secondary color as the desired bg, not black.
    #noise buffer is how exact we want the generated pixel to be compared to the ideal, ie what gets let through
    #higher number = less exact vals, but faster gen times, and more noisey noise
    des_red = des[0]
    des_green = des[1]
    des_blue = des[2]
    des_alpha = des[3]
    print("Expanding pixel with RGBA values: " + str(des_red) + " " + str(des_blue) + " " + str(des_green) + " " + str(des_alpha))
    
    #so we have our desired r value (255)???? so then we do a super inefficient while loop because im evil
    r1 = 0
    r2 = 0
    r3 = 0
    r4 = 0
    while ((r1+r2+r3+r4)/4) < (des_red - noise_buffer) or ((r1+r2+r3+r4)/4) > (des_red + noise_buffer):
        r1 = random.randint(0,255)
        r2 = random.randint(0,255)
        r3 = random.randint(0,255)
        r4 = random.randint(0,255)  
    print("Values chosen for red. Values chosen: " + str(r1) + " "+ str(r2) + " "+ str(r3) + " "+ str(r4) + ", averaging to " + str((r1+r2+r3+r4)/4) )
    
    #disgusting evil expanded code
    g1 = 0
    g2 = 0
    g3 = 0
    g4 = 0
    while ((g1+g2+g3+g4)/4) < (des_green - noise_buffer) or ((g1+g2+g3+g4)/4) > (des_green + noise_buffer):
        g1 = random.randint(0,255)
        g2 = random.randint(0,255)
        g3 = random.randint(0,255)
        g4 = random.randint(0,255)  
    print("Values chosen for green. Values chosen: " + str(g1) + " "+ str(g2) + " "+ str(g3) + " "+ str(g4) + ", averaging to " + str((g1+g2+g3+g4)/4) )
    b1 = 0
    b2 = 0
    b3 = 0
    b4 = 0
    while ((b1+b2+b3+b4)/4) < (des_blue - noise_buffer) or ((b1+b2+b3+b4)/4) > (des_blue + noise_buffer):
        b1 = random.randint(0,255)
        b2 = random.randint(0,255)
        b3 = random.randint(0,255)
        b4 = random.randint(0,255)  
    print("Values chosen for blue. Values chosen: " + str(b1) + " "+ str(b2) + " "+ str(b3) + " "+ str(b4) + ", averaging to " + str((b1+b2+b3+b4)/4) )

    return [[r1, r2, r3, r4],[g1,g2,g3,g4],[b1,b2,b3,b4]]

expand_pixel()



# #image stuffs, create our noisy/uncompressed background based on given ideal RGBA values and tolerance
original_size = 32#size of source image to be expanded
expansion_scale_factor = 2 #decompression factor, width of expanded pixel
des_size = original_size * expansion_scale_factor
input_image = Image.new(mode="RGBA", size=(des_size, des_size),
                        color="black")
pixel_map = input_image.load() #loads the image into a pixel map
width, height = input_image.size #gets the width and height of the image
for i in range(original_size):
   for j in range(original_size): 
        [[r1,r2,r3,r4],[g1,g2,g3,g4],[b1,b2,b3,b4]] = expand_pixel([25,25,25,255], 5) #change values here until i fix it and make it swag
        #working with a singular pixel0
        #top left
        pixel_map[2*i, 2*j] = (r1,g1,b1,255)
        #top right
        pixel_map[2*i+1, 2*j] = (r2,g2,b2,255)
        #bottom left
        pixel_map[2*i, 2*j+1] = (r3,g3,b3,255)
        #bottom right
        pixel_map[2*i+1, 2*j+1] = (r4,g4,b4,255)

input_image.save("reverse_compression_test_1.png", format="png") 
input_image.show()


Expanding pixel with RGBA values: 128 128 128 255
Values chosen for red. Values chosen: 245 185 75 8, averaging to 128.25
Values chosen for green. Values chosen: 75 189 19 241, averaging to 131.0
Values chosen for blue. Values chosen: 209 217 76 25, averaging to 131.75
Expanding pixel with RGBA values: 25 25 25 255
Values chosen for red. Values chosen: 0 73 2 33, averaging to 27.0
Values chosen for green. Values chosen: 35 24 3 25, averaging to 21.75
Values chosen for blue. Values chosen: 11 1 3 74, averaging to 22.25
Expanding pixel with RGBA values: 25 25 25 255
Values chosen for red. Values chosen: 22 25 56 15, averaging to 29.5
Values chosen for green. Values chosen: 44 19 25 28, averaging to 29.0
Values chosen for blue. Values chosen: 35 19 17 9, averaging to 20.0
Expanding pixel with RGBA values: 25 25 25 255
Values chosen for red. Values chosen: 13 70 3 6, averaging to 23.0
Values chosen for green. Values chosen: 63 13 0 4, averaging to 20.0
Values chosen for blue. Values chosen